In [1]:
import pyodbc
import pandas as pd
import numpy as np


In [2]:
server_name = 'identidadvoip.database.windows.net'
database_name = 'ApolloProdDb'
username = 'mfvilla'
password = 'aQ-ATd5Kk-NgVfD'



connection_string = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server_name};DATABASE={database_name};UID={username};PWD={password};LOGIN_TIMEOUT=30'
connection = pyodbc.connect(connection_string)

In [3]:
from datetime import datetime, timedelta
start_date=datetime(2024, 1, 28, 0, 0, 0)
end_date=start_date+ timedelta(days=10)

final_df = pd.DataFrame()

for _ in range(7):
  
    start_of_day = start_date.strftime('%Y-%m-%d 00:00:00')
    end_of_day = (start_date + timedelta(days=1)).strftime('%Y-%m-%d 00:00:00')

   
    query = f"""
    SELECT CDR.[CdrId], CDR.[DateCall], CDR.[SettlementPrice], CDR.[SettlementCost], CDR.[Vendor],
           VE.ContrName AS VendorName, CDR.[Client], CL.ContrName AS ClientName, CDR.[AreaNameId],
           AR.CallRoute AS Area, CDR.[TotalSeconds], CDR.[RpServId], CDR.[PrefixDescriptionDp],
           CDR.[DstBiltime], CDR.[DstBillCostAccountCur], CDR.[SrcBillCostSrcAccountCur],
           CDR.[DpBillScheme], CDR.[RpBillScheme], CDR.[ElapsedTime], CDR.[SrcAddress], CDR.[DstAddress],
           CDR.[SrcNumberIn], CDR.[DstNumberIn], CDR.[SrcNumberRoute], CDR.[DstNumberRoute],
           CDR.[SrcNumberOut], CDR.[DstNumberOut], CDR.[SrcSetupTime], CDR.[DstSetupTime],
           CDR.[SrcAlertingTime], CDR.[DstAlertingTime], CDR.[ConnectTime], CDR.[DisconnectTime],
           CDR.[DisconnectCode], CDR.[SrcDisconnectCode], CDR.[DstDisconnectCode], CDR.[AreaPrfx],
           CDR.[TechPrfx], CDR.[DstId], CDR.[DpPrfx], CDR.[DpRateDstAccountCur], CDR.[Lrn]
    FROM [dbo].[CdrBilled] CDR
    JOIN Contractors CL ON CDR.Client = CL.ContractorId
    JOIN Contractors VE ON CDR.Vendor = VE.ContractorId
    JOIN AreaNames AR ON CDR.AreaNameId = AR.AreaNameId
    WHERE CDR.DateCall >= '{start_of_day}'
    AND CDR.DateCall < '{end_of_day}'
    """

    # Read data from SQL into a DataFrame
    df_daily = pd.read_sql_query(query, connection)

    # Concatenate the daily DataFrame with the final DataFrame
    final_df = pd.concat([final_df, df_daily], ignore_index=True)

    # Move to the next day
    start_date += timedelta(days=1)

# Close the database connection
connection.close()

/tmp/ipykernel_138721/3675587145.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_daily = pd.read_sql_query(query, connection)


In [4]:
ATT=final_df[final_df['ClientName']=='AT&T Corp']

In [5]:
ATT['DateCall']=pd.to_datetime(ATT['DateCall'])

/tmp/ipykernel_138721/3199517926.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ATT['DateCall']=pd.to_datetime(ATT['DateCall'])


In [6]:
ATT=ATT.set_index('DateCall')

In [7]:
ATT['Primeros_6_numeros'] = ATT['SrcNumberRoute'].astype(str).str.slice(0, 6)
ATT['Primeros_6_numerosB'] = ATT['DstNumberRoute'].astype(str).str.slice(0, 6)

In [8]:
ATT.dtypes

CdrId                              float64
SettlementPrice                    float64
SettlementCost                     float64
Vendor                              object
VendorName                          object
Client                              object
ClientName                          object
AreaNameId                         float64
Area                                object
TotalSeconds                       float64
RpServId                           float64
PrefixDescriptionDp                 object
DstBiltime                         float64
DstBillCostAccountCur              float64
SrcBillCostSrcAccountCur           float64
DpBillScheme                        object
RpBillScheme                        object
ElapsedTime                        float64
SrcAddress                         float64
DstAddress                         float64
SrcNumberIn                         object
DstNumberIn                         object
SrcNumberRoute                      object
DstNumberRo

In [9]:
Anguilla = ATT[ATT['Area'].str.contains('Anguilla')]
BVI= ATT[ATT['Area'].str.contains('Virgin')]
Dominica=ATT[ATT['Area'].str.contains('Dominica')]

In [10]:
#Anguilla.to_excel('anguillaATTlastfebweek.xlsx')
#BVI.to_excel('BVIATTlastfebweek.xlsx')
#Dominica.to_excel('BVIATTlastfebweek.xlsx')

In [11]:
BVI

,CdrId,SettlementPrice,SettlementCost,Vendor,VendorName,Client,ClientName,AreaNameId,Area,TotalSeconds,...,SrcDisconnectCode,DstDisconnectCode,AreaPrfx,TechPrfx,DstId,DpPrfx,DpRateDstAccountCur,Lrn,Primeros_6_numeros,Primeros_6_numerosB
DateCall,,,,,,,,,,,,,,,,,,,,,
2024-01-28 00:37:45,1.170642e+17,0.18,0.160,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,4943.0,British Virgin Islands - Mobile Other,1603.0,...,131088.0,131088.0,1284499,98195#,134117.0,1284499,0.160,None,134020,128449
2024-01-28 00:12:40,1.170642e+17,0.18,0.160,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,4943.0,British Virgin Islands - Mobile Other,9.0,...,131088.0,131088.0,1284442,98195#,134113.0,1284442,0.160,None,186627,128444
2024-01-28 01:27:01,1.170642e+17,0.19,0.165,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,7061.0,British Virgin Islands - Mobile C&W,2.0,...,131088.0,131088.0,1284547,98195#,134117.0,1284547,0.165,None,180180,128454
2024-01-28 01:38:49,1.170642e+17,0.19,0.165,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,7061.0,British Virgin Islands - Mobile C&W,39.0,...,131088.0,131088.0,1284545,98195#,134113.0,1284545,0.165,None,186337,128454
2024-01-28 01:55:48,1.170642e+17,0.18,0.160,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,4943.0,British Virgin Islands - Mobile Other,99.0,...,131088.0,131088.0,1284499,98195#,134117.0,1284499,0.160,None,171887,128449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-03 23:11:47,1.170702e+17,0.19,0.165,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,7061.0,British Virgin Islands - Mobile C&W,71.0,...,131088.0,131088.0,1284547,98195#,134113.0,1284547,0.165,None,130222,128454
2024-02-03 23:27:28,1.170702e+17,0.19,0.165,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,7061.0,British Virgin Islands - Mobile C&W,73.0,...,131088.0,131088.0,1284547,98195#,134117.0,1284547,0.165,None,171897,128454
2024-02-03 23:34:56,1.170702e+17,0.16,0.160,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,2953.0,British Virgin Islands - Proper,5.0,...,131088.0,131088.0,1284,98195#,134117.0,1284869,0.160,None,171339,128486


In [12]:
import numpy as np

In [13]:
ptBVI=pd.pivot_table(BVI,index=BVI.index,columns=['Area','Primeros_6_numeros'],values='TotalSeconds',aggfunc=np.sum)

/tmp/ipykernel_138721/4059643219.py:1: FutureWarning: The provided callable <function sum at 0x7f74c033eca0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ptBVI=pd.pivot_table(BVI,index=BVI.index,columns=['Area','Primeros_6_numeros'],values='TotalSeconds',aggfunc=np.sum)


In [14]:
ptang=pd.pivot_table(Anguilla,index=Anguilla.index,columns=['Area','Primeros_6_numeros'],values='TotalSeconds',aggfunc=np.sum)

/tmp/ipykernel_138721/3396388459.py:1: FutureWarning: The provided callable <function sum at 0x7f74c033eca0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ptang=pd.pivot_table(Anguilla,index=Anguilla.index,columns=['Area','Primeros_6_numeros'],values='TotalSeconds',aggfunc=np.sum)


In [15]:
ptdom=pd.pivot_table(Dominica,index=Dominica.index,columns=['Area','Primeros_6_numeros'],values='TotalSeconds',aggfunc=np.sum)

/tmp/ipykernel_138721/2758353552.py:1: FutureWarning: The provided callable <function sum at 0x7f74c033eca0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ptdom=pd.pivot_table(Dominica,index=Dominica.index,columns=['Area','Primeros_6_numeros'],values='TotalSeconds',aggfunc=np.sum)


In [25]:
ang_codes=ptang.resample('D').sum()

In [26]:
dom_codes=ptdom.resample('D').sum()

In [ ]:
BVI_codes=ptBVI.resample('D').sum()

In [24]:
BVI_codes.to_excel('BVI_codes.xlsx')

In [27]:
dom_codes.to_excel('dom_codes.xlsx')

In [28]:
ang_codes.to_excel('ang_codes.xlsx')

In [17]:
Dominica

,CdrId,SettlementPrice,SettlementCost,Vendor,VendorName,Client,ClientName,AreaNameId,Area,TotalSeconds,...,SrcDisconnectCode,DstDisconnectCode,AreaPrfx,TechPrfx,DstId,DpPrfx,DpRateDstAccountCur,Lrn,Primeros_6_numeros,Primeros_6_numerosB
DateCall,,,,,,,,,,,,,,,,,,,,,
2024-01-28 02:09:30,1.170643e+17,0.19,0.14,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,5021.0,Dominica - Mobile C&W,16.0,...,131088.0,131088.0,1767265,98195#,134113.0,1767265,0.14,None,168241,176726
2024-01-28 03:26:58,1.170643e+17,0.19,0.14,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,5021.0,Dominica - Mobile C&W,3.0,...,131088.0,131088.0,1767245,98195#,134113.0,1767245,0.14,None,176088,176724
2024-01-28 03:27:52,1.170643e+17,0.19,0.14,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,5021.0,Dominica - Mobile C&W,7.0,...,131088.0,131088.0,1767245,98195#,134117.0,1767245,0.14,None,176088,176724
2024-01-28 06:59:36,1.170644e+17,0.19,0.14,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,5021.0,Dominica - Mobile C&W,97.0,...,131088.0,131088.0,1767235,98195#,134113.0,1767235,0.14,None,176727,176723
2024-01-28 08:15:55,1.170645e+17,0.19,0.14,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,5021.0,Dominica - Mobile C&W,3.0,...,131088.0,131088.0,1767235,98195#,134117.0,1767235,0.14,None,176729,176723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-03 21:35:56,1.170701e+17,0.19,0.14,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,5021.0,Dominica - Mobile C&W,6.0,...,131088.0,131088.0,1767225,98195#,134113.0,1767225,0.14,None,123942,176722
2024-02-03 22:06:18,1.170702e+17,0.19,0.14,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,5021.0,Dominica - Mobile C&W,11.0,...,131088.0,131088.0,1767245,98195#,134113.0,1767245,0.14,None,140433,176724
2024-02-03 22:36:06,1.170702e+17,0.19,0.14,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,5021.0,Dominica - Mobile C&W,3.0,...,131088.0,131088.0,1767295,98195#,134117.0,1767295,0.14,None,161831,176729


In [18]:
Anguilla

,CdrId,SettlementPrice,SettlementCost,Vendor,VendorName,Client,ClientName,AreaNameId,Area,TotalSeconds,...,SrcDisconnectCode,DstDisconnectCode,AreaPrfx,TechPrfx,DstId,DpPrfx,DpRateDstAccountCur,Lrn,Primeros_6_numeros,Primeros_6_numerosB
DateCall,,,,,,,,,,,,,,,,,,,,,
2024-01-28 00:36:21,1.170642e+17,0.19,0.185,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,4501.0,Anguilla - Mobile C&W,47.0,...,131088.0,131088.0,1264235,98195#,134113.0,1264235,0.185,None,126472,126423
2024-01-28 07:26:16,1.170644e+17,0.19,0.185,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,4501.0,Anguilla - Mobile C&W,1352.0,...,131088.0,131088.0,1264235,98195#,134113.0,1264235,0.185,None,148473,126423
2024-01-28 07:17:29,1.170644e+17,0.19,0.185,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,4501.0,Anguilla - Mobile C&W,21.0,...,131088.0,131088.0,1264235,98195#,134113.0,1264235,0.185,None,193162,126423
2024-01-28 07:20:18,1.170644e+17,0.19,0.185,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,4501.0,Anguilla - Mobile C&W,41.0,...,131088.0,131088.0,1264235,98195#,134117.0,1264235,0.185,None,193162,126423
2024-01-28 07:52:26,1.170645e+17,0.16,0.155,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,2862.0,Anguilla - Proper,37.0,...,131088.0,131088.0,1264,98195#,134113.0,1264497,0.155,None,156137,126449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-03 21:13:17,1.170701e+17,0.16,0.155,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,2862.0,Anguilla - Proper,74.0,...,131088.0,131088.0,1264,98195#,134113.0,1264497,0.155,None,148046,126449
2024-02-03 21:23:01,1.170701e+17,0.16,0.155,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,2862.0,Anguilla - Proper,176.0,...,131088.0,131088.0,1264,98195#,134117.0,1264497,0.155,None,165086,126449
2024-02-03 21:26:18,1.170701e+17,0.16,0.155,01.31077,Flow Jamaica Columbus Networks LLA,01.31161,AT&T Corp,2862.0,Anguilla - Proper,328.0,...,131088.0,131088.0,1264,98195#,134113.0,1264497,0.155,None,140758,126449
